In [1]:
import os
import sys


root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [3]:
from controllers.directional_trading.trend_example import TrendExampleControllerConfig
from core.backtesting.optimizer import BacktestingConfig, BaseStrategyConfigGenerator
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from decimal import Decimal


class TrendExampleConfigGenerator(BaseStrategyConfigGenerator):
    """
    Strategy configuration generator for MACD and Bollinger Bands optimization.
    """

    async def generate_config(self, trial) -> BacktestingConfig:
        # Suggest hyperparameters using the trial object
        interval = trial.suggest_categorical("interval", ["1m"])
        ema_short = trial.suggest_int("ema_short", 9, 59, step=10)
        ema_medium = trial.suggest_int("ema_medium", 10, 60, step=10)
        ema_long = trial.suggest_int("slow_ma", 21, 201, step=10)
        total_amount_quote = 1000
        max_executors_per_side = trial.suggest_int("max_executors_per_side", 1, 3)
        take_profit = trial.suggest_float("take_profit", 0.04, 0.05, step=0.01)
        stop_loss = trial.suggest_float("stop_loss", 0.01, 0.05, step=0.01)
        trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.005, 0.02, step=0.005)
        trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.1, 0.3, step=0.1)
        trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio
        time_limit = 60 * 60 * 24 * 2
        cooldown_time = 60 * 15

        # Create the strategy configuration
        config = TrendExampleControllerConfig(
            connector_name="binance_perpetual",
            trading_pair="1000BONK-USDT",
            interval=interval,
            ema_short=ema_short,
            ema_medium=ema_medium,
            ema_long=ema_long,
            total_amount_quote=Decimal(total_amount_quote),
            take_profit=Decimal(take_profit),
            stop_loss=Decimal(stop_loss),
            trailing_stop=TrailingStop(
                activation_price=Decimal(trailing_stop_activation_price),
                trailing_delta=Decimal(trailing_stop_trailing_delta),
            ),
            time_limit=time_limit,
            max_executors_per_side=max_executors_per_side,
            cooldown_time=cooldown_time,
        )

        # Return the configuration encapsulated in BacktestingConfig
        return BacktestingConfig(config=config, start=self.start, end=self.end)

In [4]:
from core.backtesting.optimizer import StrategyOptimizer
import datetime

start_date = datetime.datetime(2024, 11, 1)
end_date = datetime.datetime(2024, 11, 16)
config_generator = TrendExampleConfigGenerator(start_date=start_date, end_date=end_date)

optimizer = StrategyOptimizer(root_path=root_path, load_cached_data=True)
await optimizer.optimize(
    study_name="trend_example",
    config_generator=config_generator,
    n_trials=100,
)

[I 2024-11-21 13:20:37,967] A new study created in RDB with name: trend_example
2024-11-21 13:20:38,948 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x347448520>
2024-11-21 13:20:38,948 - asyncio - ERROR - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x3472ffa00>, 10830.093214375)])']
connector: <aiohttp.connector.TCPConnector object at 0x34744a2f0>
/Users/dman/Documents/code/quants-lab/controllers/directional_trading/trend_example.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ta.ema(length=self.config.ema_short, append=True)
/Users/dman/Documents/code/quants-lab/controllers/directional_trading/trend_example.py:71: SettingWithCopyWarning:

An error occurred during optimization: An exception is raised during the commit. This typically happens due to invalid data in the commit, e.g. exceeding max length. 


Traceback (most recent call last):
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1094, in _commit_impl
    self.engine.dialect.do_commit(self.connection)
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 686, in do_commit
    dbapi_connection.commit()
sqlite3.OperationalError: disk I/O error

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/optuna/storages/_rdb/storage.py", line 81, in _create_scoped_session
    session.commit()
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/sqlalchemy/orm/session.py", line 1454, in commit
    self._transaction.commit(_to_root=self.future)
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/sqlalchemy/orm/session.py", line 839, in commit
    trans.commit()
  File "/opt/anaconda3/envs/qua

An error occurred during optimization: An exception is raised during the commit. This typically happens due to invalid data in the commit, e.g. exceeding max length. 


Traceback (most recent call last):
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1910, in _execute_context
    self.dialect.do_execute(
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 736, in do_execute
    cursor.execute(statement, parameters)
sqlite3.DatabaseError: database disk image is malformed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/optuna/storages/_rdb/storage.py", line 80, in _create_scoped_session
    yield session
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/optuna/storages/_rdb/storage.py", line 724, in set_trial_user_attr
    self._set_trial_attr_without_commit(
  File "/opt/anaconda3/envs/quants-lab/lib/python3.10/site-packages/optuna/storages/_rdb/storage.py", line 769, in _set_trial_attr_without_commit
    s

In [ ]:

from core.backtesting.optimizer import StrategyOptimizer

optimizer.launch_optuna_dashboard()